# Find the Count of Good Integers

You are given two positive integers n and k.

An integer x is called k-palindromic if:

x is a palindrome.
x is divisible by k.
An integer is called good if its digits can be rearranged to form a k-palindromic integer. For example, for k = 2, 2020 can be rearranged to form the k-palindromic integer 2002, whereas 1010 cannot be rearranged to form a k-palindromic integer.

Return the count of good integers containing n digits.

Note that any integer must not have leading zeros, neither before nor after rearrangement. For example, 1010 cannot be rearranged to form 101.

In [ ]:
class Solution:
    def countGoodIntegers(self, n: int, k: int) -> int:
        MOD = 10**9 + 7

        # n!과 역팩토리얼을 미리 계산하는 helper 함수
        def init_factorials(n, mod):
            fact = [1] * (n + 1)
            invfact = [1] * (n + 1)
            for i in range(1, n + 1):
                fact[i] = fact[i - 1] * i % mod
            invfact[n] = pow(fact[n], mod - 2, mod)
            for i in range(n, 0, -1):
                invfact[i - 1] = invfact[i] * i % mod
            return fact, invfact

        # 멀티셋(각 digit의 사용 횟수)을 이용하여 총 순열 수를 구함
        def permutations_count(freq, total, fact, invfact):
            res = fact[total]
            for f in freq:
                res = res * invfact[f] % MOD
            return res

        fact, invfact = init_factorials(n, MOD)

        # 자리별 10의 거듭제곱을 k로 나눈 나머지를 미리 계산합니다.
        pow10 = [1] * n
        for i in range(1, n):
            pow10[i] = (pow10[i - 1] * 10) % k

        seen = set()  # 이미 처리한 멀티셋을 중복 계산하지 않기 위함
        ans = 0

        # n이 짝수인 경우: 전체 자릿수 = 2L, 왼쪽 L자리를 결정하면 오른쪽은 거울 대칭으로 결정됨.
        if n % 2 == 0:
            L = n // 2

            def dfs(pos, left_digits, current_mod):
                nonlocal ans
                if pos == L:
                    # 왼쪽 절반의 각 자리들은 전체 자리에서의 기여도가 다름:
                    # 왼쪽 : 각 자리가 위치 n-1-pos
                    # 오른쪽 거울 : 각 자리가 위치 pos (즉, pow10[pos]의 자릿수)
                    mirror_mod = 0
                    for i in range(L):
                        mirror_mod = (mirror_mod + left_digits[i] * pow10[i]) % k
                    full_mod = (current_mod + mirror_mod) % k
                    if full_mod % k == 0:
                        freq = [0] * 10
                        for d in left_digits:
                            freq[d] += 2  # 왼쪽과 오른쪽에 동일하게 배정됨.
                        freq = tuple(freq)
                        if freq not in seen:
                            seen.add(freq)
                            total_perm = permutations_count(freq, n, fact, invfact)
                            # 선행 0: 첫 자리가 0인 순열 제거
                            if freq[0] > 0:
                                freq_list = list(freq)
                                freq_list[0] -= 1
                                invalid = permutations_count(tuple(freq_list), n - 1, fact, invfact)
                            else:
                                invalid = 0
                            ans = (ans + (total_perm - invalid) % MOD) % MOD
                    return

                for d in range(10):
                    if pos == 0 and d == 0:
                        continue  # 왼쪽 절반의 첫 자리는 0이면 안 됨.
                    # 왼쪽 자릿수의 기여도: 현재 전체 수의 인덱스는 n-1-pos
                    new_mod = (current_mod + d * pow10[n - 1 - pos]) % k
                    dfs(pos + 1, left_digits + [d], new_mod)

            dfs(0, [], 0)

        # n이 홀수인 경우: 전체 자릿수 = 2L+1, 왼쪽 L자리를 결정한 뒤 중앙자리 선택 → 오른쪽은 거울 대칭.
        else:
            L = n // 2

            def dfs(pos, left_digits, current_mod):
                nonlocal ans
                if pos == L:
                    for center in range(10):
                        # 중앙 자리의 기여도: 위치 L의 자릿수는 pow10[L] (n=2L+1)
                        center_contrib = center * pow10[L] % k
                        mirror_mod = 0
                        for i in range(L):
                            mirror_mod = (mirror_mod + left_digits[i] * pow10[i]) % k
                        total_mod = (current_mod + center_contrib + mirror_mod) % k
                        if total_mod % k == 0:
                            freq = [0] * 10
                            for d in left_digits:
                                freq[d] += 2
                            freq[center] += 1
                            freq = tuple(freq)
                            if freq not in seen:
                                seen.add(freq)
                                total_perm = permutations_count(freq, n, fact, invfact)
                                if freq[0] > 0:
                                    freq_list = list(freq)
                                    freq_list[0] -= 1
                                    invalid = permutations_count(tuple(freq_list), n - 1, fact, invfact)
                                else:
                                    invalid = 0
                                ans = (ans + (total_perm - invalid) % MOD) % MOD
                    return

                for d in range(10):
                    if pos == 0 and d == 0:
                        continue
                    # 홀수인 경우, 왼쪽 자릿수의 기여도: 위치는 n-1-pos (n = 2L+1)
                    new_mod = (current_mod + d * pow10[n - 1 - pos]) % k
                    dfs(pos + 1, left_digits + [d], new_mod)

            dfs(0, [], 0)

        return ans % MOD
